# Geração de bases modeladas para exploração de dados no Power BI

O dicionário de campos da base de dados do SINASC se encontra nesta [planilha](https://docs.google.com/spreadsheets/d/1TwysowkLCLIGGLplKkKibahrsG52NqVKOZuTjWyrZ-8/edit#gid=1412172863).

# 📚 Libs

In [1]:
import pandas as pd
import numpy as np
import re #para expressões regulares
from datetime import datetime

# 🎲 FATO SINASC

[Fonte OpenDataSus](https://opendatasus.saude.gov.br/dataset/sistema-de-informacao-sobre-nascidos-vivos-sinasc)

## Importação

In [2]:
arquivo = r'C:\Users\carol\OneDrive\Estudos\MBA Data Science\TCC\SINASC\Dados_SINASC\SINASC_2022.csv'
dados = pd.read_csv(arquivo, sep=";")

C:\Users\carol\AppData\Local\Temp\ipykernel_12912\2363134596.py:2: DtypeWarning: Columns (32) have mixed types. Specify dtype option on import or set low_memory=False.
  dados = pd.read_csv(arquivo, sep=";")


## Limpeza dos dados

In [3]:
#mantendo colunas que vamos usar
colunas_selecionadas = ['DTNASC',
'LOCNASC',
'CODMUNNASC',
'CODESTAB',
'SEXO',
'RACACOR',
'PESO',
'CODANOMAL',
'IDANOMAL',
'GESTACAO',
'SEMAGESTAC',
'GRAVIDEZ',
'TPAPRESENT',
'STTRABPART',
'STCESPARTO',
'PARTO',
'TPNASCASSI',
'IDADEMAE',
'ESTCIVMAE',
'RACACORMAE',
'ESCMAE',
'ESCMAE2010',
'ESCMAEAGR1',
'CODOCUPMAE',
'PARIDADE',
'QTDGESTANT',
'QTDFILVIVO',
'QTDFILMORT',
'QTDPARTNOR',
'QTDPARTCES',
'MESPRENAT',
'CONSULTAS',
'CONSPRENAT',
'KOTELCHUCK',
'TPROBSON',
'IDADEPAI']

dados = dados[colunas_selecionadas]

## Transformação de colunas

### DTNASC

In [4]:
dados['DTNASC'] = dados['DTNASC'].astype(str).str.pad(8, side = 'left', fillchar = '0')
dados['DTNASC'] = pd.to_datetime(dados['DTNASC'], format='%d%m%Y')

### CODANOMAL

In [5]:
#substitui a letra X por nada, pois não encontramos anomalia contendo a letra X no fim
dados['CODANOMAL'] = dados['CODANOMAL'].str.replace('X', '')
#substituindo o CID Q356 por Q359 pois foi substituído
#https://www.hidoctor.com.br/cid10/p/capitulo/17/grupo/Q35-Q37/categoria/Q35/subcategoria/Q359
dados['CODANOMAL'] = dados['CODANOMAL'].str.replace('Q356', 'Q359')
# conta quantas letras possui no campo CODANOMAL
dados['qt_anomal'] = dados['CODANOMAL'].str.count('[a-zA-Z]')
#substitui valores nulos e aplica formato de número inteiro
dados['qt_anomal'] = dados['qt_anomal'].fillna(0).astype(int)

### CODESTAB

In [6]:
dados['CODESTAB'] = dados['CODESTAB'].fillna(0).astype(int).astype(str)

## Criação de novas colunas

### indice

In [7]:
#coluna adicional de índice
dados['indice'] = range(1, len(dados) + 1)
dados['indice'] = dados['indice'].astype(str)

### ano_mes

In [8]:
dados['ano_mes'] = dados['DTNASC'].dt.strftime('%Y-%m')

### atualização colunas

In [9]:
#Renomeando todas colunas
novo_nome = {
    'DTNASC': 'dt_nasc',
    'LOCNASC': 'loc_nasc',
    'CODMUNNASC': 'cod_mun_nasc',
    'CODESTAB': 'cod_estab',
    'SEXO': 'sexo',
    'RACACOR': 'raca_cor',
    'PESO': 'peso',
    'CODANOMAL': 'cod_anomal',
    'IDANOMAL': 'id_anomal',
    'GESTACAO': 'gestacao',
    'SEMAGESTAC': 'sema_gestac',
    'GRAVIDEZ': 'gravidez',
    'TPAPRESENT': 'tpa_present',
    'STTRABPART': 'st_trab_parto',
    'STCESPARTO': 'st_ces_parto',
    'PARTO': 'parto',
    'TPNASCASSI': 'tp_nasc_assi',
    'IDADEMAE': 'idade_mae',
    'ESTCIVMAE': 'est_civ_mae',
    'RACACORMAE': 'raca_cor_mae',
    'ESCMAE': 'esc_mae',
    'ESCMAE2010': 'esc_mae_2010',
    'ESCMAEAGR1': 'esc_mae_gr1',
    'CODOCUPMAE': 'cod_ocup_mae',
    'PARIDADE': 'paridade',
    'QTDGESTANT': 'qtd_gestant',
    'QTDFILVIVO': 'qtd_fil_vivo',
    'QTDFILMORT': 'qtd_fil_mort',
    'QTDPARTNOR': 'qtd_part_nor',
    'QTDPARTCES': 'qtd_part_ces',
    'MESPRENAT': 'mes_pre_nat',
    'CONSULTAS': 'consultas',
    'CONSPRENAT': 'cons_pre_nat',
    'KOTELCHUCK': 'kotelchuck',
    'TPROBSON': 'tp_robson',
    'IDADEPAI': 'idade_pai'
}

dados.rename(columns=novo_nome, inplace=True)

In [10]:
#transformando colunas float que na verdade são string
colunas_float = ['raca_cor', 'cod_anomal', 'id_anomal', 'gestacao', 'gravidez', 'tpa_present', 'st_trab_parto', 'st_ces_parto', 'parto', 'tp_nasc_assi', 'est_civ_mae', 'raca_cor_mae', 'esc_mae', 'esc_mae_2010', 'cod_ocup_mae', 'qtd_gestant', 'qtd_fil_vivo', 'qtd_fil_mort', 'qtd_part_nor', 'qtd_part_ces', 'mes_pre_nat', 'consultas', 'cons_pre_nat', 'sema_gestac']

for coluna in colunas_float:
    dados [coluna] = dados[coluna].astype(str).str.replace('.0', '')

In [11]:
#transformando coluna que está como float mas é int
coluna_int = ['idade_mae', 'idade_pai']
for coluna in coluna_int:
    dados [coluna] = dados[coluna].fillna(0).astype(int)

In [12]:
dados.head()

,dt_nasc,loc_nasc,cod_mun_nasc,cod_estab,sexo,raca_cor,peso,cod_anomal,id_anomal,gestacao,...,qtd_part_ces,mes_pre_nat,consultas,cons_pre_nat,kotelchuck,tp_robson,idade_pai,qt_anomal,indice,ano_mes
0,2022-01-03,1,110001,2516500,2,1,3412.0,nan,2,5,...,0,2,4,7,5,7,20,0,1,2022-01
1,2022-01-07,1,110001,2516500,2,1,3594.0,nan,2,5,...,0,2,4,8,5,1,28,0,2,2022-01
2,2022-01-12,1,110001,2516500,2,nan,3470.0,nan,2,5,...,1,1,4,7,5,5,37,0,3,2022-01
3,2022-01-12,1,110001,2516500,2,4,3260.0,nan,2,5,...,1,1,4,31,5,5,40,0,4,2022-01
4,2022-01-17,1,110001,2516500,2,nan,3232.0,nan,2,5,...,0,2,4,8,5,2,37,0,5,2022-01


## csv

In [13]:
dados.to_csv('f_sinasc_tratado', sep=';', index=False)

# 🎲 DIMENSÃO ANOMALIAS

In [14]:
df_cod_anomal_sep = pd.DataFrame({'cod_anomal': dados['cod_anomal'].unique()})
#quebra os códigos de anomalias
df_cod_anomal_sep['cod_anomal_sep'] =  df_cod_anomal_sep['cod_anomal'].str.findall(r'[a-zA-Z]+\d+')

In [15]:
df_cod_anomal_sep = df_cod_anomal_sep.explode('cod_anomal_sep')

In [16]:
df_cod_anomal_sep = df_cod_anomal_sep[df_cod_anomal_sep.cod_anomal.notnull()]
df_cod_anomal_sep = df_cod_anomal_sep[df_cod_anomal_sep['cod_anomal'] != 'nan']

In [17]:
df_cod_anomal_sep.head()

,cod_anomal,cod_anomal_sep
1,Q369,Q369
2,Q758,Q758
3,Q699Q709,Q699
3,Q699Q709,Q709
4,Q564,Q564


In [18]:
df_cod_anomal_sep.info()

<class 'pandas.core.frame.DataFrame'>
Index: 9996 entries, 1 to 3839
Data columns (total 2 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   cod_anomal      9996 non-null   object
 1   cod_anomal_sep  9996 non-null   object
dtypes: object(2)
memory usage: 234.3+ KB


In [19]:
df_cod_anomal_sep.to_csv('d_anomalias.csv', sep=';', index=False)

# 🎲 DIMENSÃO DESCRIÇÃO ANOMALIAS
[Fonte Datasus](http://www2.datasus.gov.br/cid10/V2008/descrcsv.htm)

In [20]:
cols = ['SUBCAT', 'CLASSIF', 'RESTRSEXO', 'CAUSAOBITO', 'DESCRICAO']
planilha_subcategoria_anomalias = r'C:\Users\carol\OneDrive\Estudos\MBA Data Science\TCC\SINASC\CID-10-SUBCATEGORIAS.csv'
df_anomal = pd.read_csv(planilha_subcategoria_anomalias, usecols= cols, sep =';', encoding='ISO-8859-1')

In [21]:
#renomeando colunas
novo_nome = {
    'SUBCAT': 'subcat',
    'CLASSIF': 'classif',
    'RESTRSEXO': 'restr_sexo',
    'CAUSAOBITO': 'causa_obito',
    'DESCRICAO': 'descricao_subcat',
    'DESCRABREV': 'desc_abrev_subcat'
}

df_anomal.rename(columns=novo_nome, inplace=True)

In [22]:
#criando a chave de categoria para facilitar a análise
df_anomal['cat'] = df_anomal['subcat'].str.slice(0,3)

In [23]:
#importando base de categorias
cols = ['CAT', 'DESCRICAO']
planilha_categoria_anomalias = r'C:\Users\carol\OneDrive\Estudos\MBA Data Science\TCC\SINASC\CID-10-CATEGORIAS.csv'
df_cat_anomal = pd.read_csv(planilha_categoria_anomalias, usecols= cols, sep =';', encoding='ISO-8859-1')

In [24]:
#renomeando colunas
novo_nome = {
    'CAT': 'cat',
    'DESCRICAO': 'descricao_cat'
}

df_cat_anomal.rename(columns=novo_nome, inplace=True)

In [25]:
#fazendo o join entre subcategoria e categoria
df_anomal = pd.merge(df_anomal,df_cat_anomal, on=['cat'], how = 'left')

In [26]:
df_anomal.head()

,subcat,classif,restr_sexo,causa_obito,descricao_subcat,cat,descricao_cat
0,A000,NaN,NaN,NaN,"Cólera devida a Vibrio cholerae 01, biótipo ch...",A00,Cólera
1,A001,NaN,NaN,NaN,"Cólera devida a Vibrio cholerae 01, biótipo El...",A00,Cólera
2,A009,NaN,NaN,NaN,Cólera não especificada,A00,Cólera
3,A010,NaN,NaN,NaN,Febre tifóide,A01,Febres tifóide e paratifóide
4,A011,NaN,NaN,NaN,Febre paratifóide A,A01,Febres tifóide e paratifóide


In [27]:
df_anomal.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12451 entries, 0 to 12450
Data columns (total 7 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   subcat            12451 non-null  object
 1   classif           378 non-null    object
 2   restr_sexo        874 non-null    object
 3   causa_obito       1291 non-null   object
 4   descricao_subcat  12451 non-null  object
 5   cat               12451 non-null  object
 6   descricao_cat     12451 non-null  object
dtypes: object(7)
memory usage: 681.0+ KB


In [28]:
df_anomal.to_csv('d_desc_anomal.csv', sep = ';', index=False)

# 🎲 DIMENSÃO REGIÃO IMEDIATA

[Fonte IBGE Censo 2022 - Composição das Regiões Geográficas Imediatas](https://www.ibge.gov.br/estatisticas/sociais/populacao/22827-censo-demografico-2022.html?edicao=38166&t=resultados)

In [29]:
planilha_regiao_imediata = r'C:\Users\carol\OneDrive\Estudos\MBA Data Science\TCC\SINASC\CD2022_Composicao_Regioes_Geograficas_Imediatas.xlsx'
df_regiao_imediata = pd.read_excel(planilha_regiao_imediata, sheet_name=0, usecols='B:E', skiprows=2)

In [30]:
df_regiao_imediata.columns = ['geocod_imediata', 'nome_geo_imediata', 'geocod_muni', 'nome_muni']

In [31]:
df_regiao_imediata['nome_geo_imediata'] = df_regiao_imediata['nome_geo_imediata'].str.replace("'", "`")

In [32]:
df_regiao_imediata = df_regiao_imediata.astype('object')

In [33]:
#trata columa de codificação
df_regiao_imediata['geocod_muni'] = df_regiao_imediata['geocod_muni'].astype(str).str.strip()

In [34]:
df_regiao_imediata.head()

,geocod_imediata,nome_geo_imediata,geocod_muni,nome_muni
0,110001,Porto Velho,1100106,Guajará-Mirim
1,110001,Porto Velho,1100205,Porto Velho
2,110001,Porto Velho,1100338,Nova Mamoré
3,110001,Porto Velho,1100809,Candeias do Jamari
4,110001,Porto Velho,1101104,Itapuã do Oeste


In [35]:
df_regiao_imediata.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5570 entries, 0 to 5569
Data columns (total 4 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   geocod_imediata    5570 non-null   object
 1   nome_geo_imediata  5570 non-null   object
 2   geocod_muni        5570 non-null   object
 3   nome_muni          5570 non-null   object
dtypes: object(4)
memory usage: 174.2+ KB


# 🎲 DIMENSAO MUNICIPIO

[Fonte IBGE](https://www.ibge.gov.br/explica/codigos-dos-municipios.php)

In [36]:
#intervalo2 = pd.read_excel(url_planilha, sheet_name='emissoes_C02', usecols= 'A:D', nrows=10)
planilha_muni = r'C:\Users\carol\OneDrive\Estudos\MBA Data Science\TCC\SINASC\RELATORIO_DTB_BRASIL_MUNICIPIO.xls'

cols = ['UF', 'Nome_UF', 'Município' ,'Código Município Completo', 'Nome_Município']
df_muni = pd.read_excel(planilha_muni, sheet_name='DTB_2022_Municipio', usecols= cols, skiprows=6, nrows=5577)

WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
*** No CODEPAGE record, no encoding_override: will use 'iso-8859-1'


In [37]:
#renomeando colunas
novo_nome = {
    'UF': 'cod_uf',
    'Nome_UF': 'uf',
    'Município': 'cod_muni',
    'Código Município Completo': 'cod_muni_completo',
    'Nome_Município': 'nome_muni'
}

df_muni.rename(columns=novo_nome, inplace=True)

In [38]:
#transformando col de cod para object
col_object1 = ['cod_uf', 'cod_muni', 'cod_muni_completo']
df_muni[col_object1] = df_muni[col_object1].astype(str)    

In [39]:
#transformando chave muni
df_muni['cod_muni'] = df_muni['cod_muni'].astype(str).str.pad(5, side = 'left', fillchar = '0')
#retirando 1 caracter da coluna cod_municompleto
df_muni['cod_muni_completo'] = df_muni['cod_muni_completo'].str.slice(0,6)

In [40]:
#criando a coluna chave_pop
df_muni['chave_pop'] = df_muni['cod_uf'].astype(str) + df_muni['cod_muni'].astype(str)

In [41]:
#criando coluna de regiao

# mapeia os estados para suas respectivas regiões
regiao_por_estado = {
    'Norte': ['Acre', 'Amapá', 'Amazonas', 'Pará', 'Rondônia', 'Roraima', 'Tocantins'],
    'Nordeste': ['Alagoas', 'Bahia', 'Ceará', 'Maranhão', 'Paraíba', 'Pernambuco', 'Piauí', 'Rio Grande do Norte', 'Sergipe'],
    'Centro-Oeste': ['Distrito Federal', 'Goiás', 'Mato Grosso', 'Mato Grosso do Sul'],
    'Sudeste': ['Espírito Santo', 'Minas Gerais', 'Rio de Janeiro', 'São Paulo'],
    'Sul': ['Paraná', 'Rio Grande do Sul', 'Santa Catarina']
}

# Função para mapear estados para regiões
def mapear_regiao(uf):
    for regiao, estados in regiao_por_estado.items():
        if uf in estados:
            return regiao
    return None  # Retorna None se o estado não for encontrado

#aplica na coluna
df_muni['regiao'] = df_muni['uf'].apply(mapear_regiao)

In [42]:
df_muni['chave_pop'] = df_muni['chave_pop'].astype(str).str.strip()
#para garantir formato de coluna para join futuro

In [43]:
df_muni.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5570 entries, 0 to 5569
Data columns (total 7 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   cod_uf             5570 non-null   object
 1   uf                 5570 non-null   object
 2   cod_muni           5570 non-null   object
 3   cod_muni_completo  5570 non-null   object
 4   nome_muni          5570 non-null   object
 5   chave_pop          5570 non-null   object
 6   regiao             5570 non-null   object
dtypes: object(7)
memory usage: 304.7+ KB


In [44]:
df_muni.head()


,cod_uf,uf,cod_muni,cod_muni_completo,nome_muni,chave_pop,regiao
0,11,Rondônia,00015,110001,Alta Floresta D'Oeste,1100015,Norte
1,11,Rondônia,00379,110037,Alto Alegre dos Parecis,1100379,Norte
2,11,Rondônia,00403,110040,Alto Paraíso,1100403,Norte
3,11,Rondônia,00346,110034,Alvorada D'Oeste,1100346,Norte
4,11,Rondônia,00023,110002,Ariquemes,1100023,Norte


In [45]:
#agregando os dados de região imediata aos dados do município
df_muni = df_muni.merge(df_regiao_imediata[['geocod_muni', 'geocod_imediata', 'nome_geo_imediata']], 
                                  left_on='chave_pop', right_on='geocod_muni', how='left')

In [46]:
colunas_finais_ordenadas = [
    'regiao', 
    'cod_uf', 
    'uf', 
    'geocod_imediata', 
    'nome_geo_imediata',
    'cod_muni', 
    'cod_muni_completo', 
    'chave_pop', 
    'nome_muni'
]
df_muni = df_muni[colunas_finais_ordenadas]

In [47]:
df_muni.head(5)

,regiao,cod_uf,uf,geocod_imediata,nome_geo_imediata,cod_muni,cod_muni_completo,chave_pop,nome_muni
0,Norte,11,Rondônia,110005,Cacoal,00015,110001,1100015,Alta Floresta D'Oeste
1,Norte,11,Rondônia,110005,Cacoal,00379,110037,1100379,Alto Alegre dos Parecis
2,Norte,11,Rondônia,110002,Ariquemes,00403,110040,1100403,Alto Paraíso
3,Norte,11,Rondônia,110004,Ji-Paraná,00346,110034,1100346,Alvorada D'Oeste
4,Norte,11,Rondônia,110002,Ariquemes,00023,110002,1100023,Ariquemes


In [48]:
df_muni.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5570 entries, 0 to 5569
Data columns (total 9 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   regiao             5570 non-null   object
 1   cod_uf             5570 non-null   object
 2   uf                 5570 non-null   object
 3   geocod_imediata    5570 non-null   object
 4   nome_geo_imediata  5570 non-null   object
 5   cod_muni           5570 non-null   object
 6   cod_muni_completo  5570 non-null   object
 7   chave_pop          5570 non-null   object
 8   nome_muni          5570 non-null   object
dtypes: object(9)
memory usage: 391.8+ KB


## 🗺️ Dados de latlong

In [49]:
url_latlong_csv = 'https://raw.githubusercontent.com/kelvins/municipios-brasileiros/refs/heads/main/csv/municipios.csv'
df_latlong = pd.read_csv(url_latlong_csv)

In [50]:
df_latlong.head(5)

,codigo_ibge,nome,latitude,longitude,capital,codigo_uf,siafi_id,ddd,fuso_horario
0,5200050,Abadia de Goiás,-16.75730,-49.4412,0,52,1050,62,America/Sao_Paulo
1,3100104,Abadia dos Dourados,-18.48310,-47.3916,0,31,4001,34,America/Sao_Paulo
2,5200100,Abadiânia,-16.19700,-48.7057,0,52,9201,62,America/Sao_Paulo
3,3100203,Abaeté,-19.15510,-45.4444,0,31,4003,37,America/Sao_Paulo
4,1500107,Abaetetuba,-1.72183,-48.8788,0,15,401,91,America/Sao_Paulo


In [51]:
df_latlong.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5570 entries, 0 to 5569
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   codigo_ibge   5570 non-null   int64  
 1   nome          5570 non-null   object 
 2   latitude      5570 non-null   float64
 3   longitude     5570 non-null   float64
 4   capital       5570 non-null   int64  
 5   codigo_uf     5570 non-null   int64  
 6   siafi_id      5570 non-null   int64  
 7   ddd           5570 non-null   int64  
 8   fuso_horario  5570 non-null   object 
dtypes: float64(2), int64(5), object(2)
memory usage: 391.8+ KB


In [52]:
#pegando as colunas necessárias e tratando a coluna do ibge para merge com d_muni
df_latlong_trat = df_latlong[['codigo_ibge', 'latitude', 'longitude']].assign(
    codigo_ibge=df_latlong['codigo_ibge'].astype(str).str[:6]
)

In [53]:
df_muni_latlong = df_muni.merge(
    df_latlong_trat[['codigo_ibge', 'latitude', 'longitude']],
    left_on='cod_muni_completo',
    right_on='codigo_ibge',
    how='left'
)

In [54]:
df_muni_latlong.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5570 entries, 0 to 5569
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   regiao             5570 non-null   object 
 1   cod_uf             5570 non-null   object 
 2   uf                 5570 non-null   object 
 3   geocod_imediata    5570 non-null   object 
 4   nome_geo_imediata  5570 non-null   object 
 5   cod_muni           5570 non-null   object 
 6   cod_muni_completo  5570 non-null   object 
 7   chave_pop          5570 non-null   object 
 8   nome_muni          5570 non-null   object 
 9   codigo_ibge        5570 non-null   object 
 10  latitude           5570 non-null   float64
 11  longitude          5570 non-null   float64
dtypes: float64(2), object(10)
memory usage: 522.3+ KB


In [55]:
df_muni_latlong.head(5)

,regiao,cod_uf,uf,geocod_imediata,nome_geo_imediata,cod_muni,cod_muni_completo,chave_pop,nome_muni,codigo_ibge,latitude,longitude
0,Norte,11,Rondônia,110005,Cacoal,00015,110001,1100015,Alta Floresta D'Oeste,110001,-11.92830,-61.9953
1,Norte,11,Rondônia,110005,Cacoal,00379,110037,1100379,Alto Alegre dos Parecis,110037,-12.13200,-61.8350
2,Norte,11,Rondônia,110002,Ariquemes,00403,110040,1100403,Alto Paraíso,110040,-9.71429,-63.3188
3,Norte,11,Rondônia,110004,Ji-Paraná,00346,110034,1100346,Alvorada D'Oeste,110034,-11.34630,-62.2847
4,Norte,11,Rondônia,110002,Ariquemes,00023,110002,1100023,Ariquemes,110002,-9.90571,-63.0325


In [56]:
df_muni_latlong.to_csv('d_muni_latlong.csv', sep = ';', index=False)

# 🎲 DIMENSÃO POPULAÇÃO RESIDENTE POR SEXO

[Fonte IBGE Censo 2022](https://www.ibge.gov.br/estatisticas/sociais/populacao/22827-censo-demografico-2022.html?edicao=38166&t=resultados)

Através da pesquisa [dos municípios que compõe a região imediata Valença do Rio de Janeiro e Bahia e Região Itabaiana da Paraíba e Sergipe](https://www.ibge.gov.br/estatisticas/sociais/populacao/22827-censo-demografico-2022.html?edicao=38166&t=resultados) no [levantamento de estimativa de População por Município de 2024 do IBGE](https://www.ibge.gov.br/estatisticas/sociais/populacao/9103-estimativas-de-populacao.html?edicao=41105&t=resultados) temos que:
| geocod_imediata | nome_geo_imediata | uf            | estimativa_pop_2024 | pop_residente_2022 |
|---------------------|-------------------|---------------|---------------------|--------------------------|
| 250004              | Itabaiana          | Paraíba       | 64.875              | 63.422                   |
| 280004              | Itabaiana          | Sergipe       | 241.769             | 252.041                  |
| 290005              | Valença            | Bahia         | 236.448             | 226.779                  |
| 330006              | Valença            | Rio de Janeiro| 176.098             | 167.219                  |


In [57]:
def trata_dados_populacao(file_path, usecols, sexo):
    # Importar planilha
    df = pd.read_excel(file_path, sheet_name=0, usecols=usecols, skiprows=5)
    
    # Renomear a coluna de 'Região Geográfica Imediata' para nome_geo_imediata
    df.columns = ['nome_geo_imediata'] + df.columns[1:].tolist()
    
    # Substituir caractere ' por ` para uniformidade
    df['nome_geo_imediata'] = df['nome_geo_imediata'].str.replace("'", "´")


    # Função para definir a coluna geo_cod_imediata_setada
    def set_geo_cod_imediata(row):
        if row.name == 135:
            return 250004
        elif row.name == 179:
            return 280004
        elif row.name == 186:
            return 290005
        elif row.name == 299:
            return 330006
        else:
            return None
        
    df['geocod_imediata'] = df.apply(set_geo_cod_imediata, axis=1)
    df['geocod_imediata'] = df['geocod_imediata'].fillna(0)
    df['geocod_imediata'] = df['geocod_imediata'].astype(str).str.replace('.0', '', regex=False).astype(int)
    
    # Transformar os dados (descer as colunas de faixa_etária)
    df_long = df.melt(id_vars=['nome_geo_imediata', 'geocod_imediata'], var_name='faixa_idade', value_name='pop_residente')
    
    # Ajustar a coluna faixa_idade
    df_long['faixa_idade'] = df_long['faixa_idade'].str.replace(r' anos.*', '', regex=True)

    # Preencher e ajustar os valores de pop_residente
    df_long['pop_residente'] = df_long['pop_residente'].fillna(0)
    df_long['pop_residente'] = df_long['pop_residente'].replace('-', '0')
    df_long['pop_residente'] = df_long['pop_residente'].astype(str).str.replace('.0', '', regex=False).astype(int)

    # Adicionar colunas extras de ano e sexo
    df_long['ano'] = 2022
    df_long['sexo'] = sexo
    
    # Reordenar colunas
    df_long = df_long[['ano', 'sexo', 'nome_geo_imediata', 'geocod_imediata', 'faixa_idade', 'pop_residente']]
    
    return df_long

In [58]:
planilha_censo_2022 = r'C:\Users\carol\OneDrive\Estudos\MBA Data Science\TCC\SINASC\tabela9514_grupo_idade_RGI.xlsx'

In [59]:
homens_long = trata_dados_populacao(planilha_censo_2022, 'A,Z:AT', 'M')
mulheres_long = trata_dados_populacao(planilha_censo_2022, 'A,AV:BP', 'F')
pop_residente_sexo = pd.concat([homens_long, mulheres_long])
pop_residente_sexo = pop_residente_sexo[pop_residente_sexo['nome_geo_imediata'] != 'Fonte: IBGE - Censo Demográfico']
pop_residente_sexo = pop_residente_sexo[['ano', 'sexo', 'nome_geo_imediata', 'faixa_idade', 'pop_residente', 'geocod_imediata']]

In [60]:
#quebrando os dfs para o tratamento dos duplicados:
pop_residente_sexo_setado = pop_residente_sexo[pop_residente_sexo['nome_geo_imediata'].isin(['Valença', 'Itabaiana'])]

pop_residente_sexo_geral = pop_residente_sexo[
    ~pop_residente_sexo['nome_geo_imediata'].isin(['Valença', 'Itabaiana'])
][['ano', 'sexo', 'nome_geo_imediata', 'faixa_idade', 'pop_residente']]

In [61]:
pop_residente_sexo_geral = pop_residente_sexo_geral.merge(df_regiao_imediata[['nome_geo_imediata', 'geocod_imediata']].drop_duplicates(),
                                                          on='nome_geo_imediata', how='left')

In [62]:
pop_residente_sexo = pd.concat([pop_residente_sexo_geral, pop_residente_sexo_setado])

In [63]:
pop_residente_sexo.info()

<class 'pandas.core.frame.DataFrame'>
Index: 21420 entries, 0 to 10519
Data columns (total 6 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   ano                21420 non-null  int64 
 1   sexo               21420 non-null  object
 2   nome_geo_imediata  21420 non-null  object
 3   faixa_idade        21420 non-null  object
 4   pop_residente      21420 non-null  int32 
 5   geocod_imediata    21420 non-null  object
dtypes: int32(1), int64(1), object(4)
memory usage: 1.1+ MB


In [64]:
pop_residente_sexo.head(5)

,ano,sexo,nome_geo_imediata,faixa_idade,pop_residente,geocod_imediata
0,2022,M,Porto Velho,0 a 4,20620,110001
1,2022,M,Ariquemes,0 a 4,6877,110002
2,2022,M,Jaru,0 a 4,3868,110003
3,2022,M,Ji-Paraná,0 a 4,10402,110004
4,2022,M,Cacoal,0 a 4,10202,110005


In [65]:
colunas_finais_ordenadas = [
    'ano',
    'sexo',
    'geocod_imediata',
    'faixa_idade',
    'pop_residente'
]
pop_residente_sexo = pop_residente_sexo[colunas_finais_ordenadas]

In [66]:
pop_residente_sexo.head(5)

,ano,sexo,geocod_imediata,faixa_idade,pop_residente
0,2022,M,110001,0 a 4,20620
1,2022,M,110002,0 a 4,6877
2,2022,M,110003,0 a 4,3868
3,2022,M,110004,0 a 4,10402
4,2022,M,110005,0 a 4,10202


In [67]:
pop_residente_sexo.to_csv('d_pop_residente.csv', sep = ';', index=False)

# 🎲 DIMENSÃO POPULAÇÃO

[Fonte IBGE](https://www.ibge.gov.br/estatisticas/sociais/populacao/9103-estimativas-de-populacao.html?edicao=31451)

In [68]:
planilha_pop = r'C:\Users\carol\OneDrive\Estudos\MBA Data Science\TCC\SINASC\POP2021_20230710.xls'
cols = ['COD. UF', 'COD. MUNIC', 'POPULAÇÃO ESTIMADA']
df_pop = pd.read_excel(planilha_pop, sheet_name='Municípios', usecols= cols, skiprows=1, nrows=5572)

In [69]:
#renomeando colunas
novo_nome = {
    'COD. UF': 'cod_uf',
    'COD. MUNIC': 'cod_muni',
    'POPULAÇÃO ESTIMADA': 'pop_estimada'
}

df_pop.rename(columns=novo_nome, inplace=True)

In [70]:
#alterando tipo de colunas
#transformando col de cod para object
col_object1 = ['cod_uf', 'cod_muni']
df_pop[col_object1] = df_pop[col_object1].fillna(0).astype(int).astype(str)

In [71]:
#acrescentando zeros à esquerda do código do municipio
df_pop['cod_muni'] = df_pop['cod_muni'].str.pad(5, side = 'left', fillchar = '0')

In [72]:
#tirando . e ou vírgula dos campos
def limpar_string(texto):
    # Verificar se o valor é uma string
    if isinstance(texto, str):
        # Define o padrão regex para encontrar parênteses e seu conteúdo
        padrao = r'\([^)]*\)'
        
        # Substitui o padrão por uma string vazia
        texto_limpo = re.sub(padrao, '', texto)
        
        # Remove pontos e vírgulas
        texto_limpo = texto_limpo.replace('.', '').replace(',', '')
        
        return texto_limpo
    else:
        return texto

df_pop['pop_estimada'] = df_pop['pop_estimada'].apply(limpar_string)

In [73]:
#transformando a coluna de população para int
df_pop['pop_estimada'] = df_pop['pop_estimada'].fillna(0).astype(int)

In [74]:
#criando nova coluna chave
df_pop['chave_pop'] = df_pop['cod_uf'].astype(str) + df_pop['cod_muni'].astype(str)

In [75]:
df_pop.head()

,cod_uf,cod_muni,pop_estimada,chave_pop
0,11,00015,22516,1100015
1,11,00023,111148,1100023
2,11,00031,5067,1100031
3,11,00049,86416,1100049
4,11,00056,16088,1100056


In [76]:
df_pop.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5572 entries, 0 to 5571
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   cod_uf        5572 non-null   object
 1   cod_muni      5572 non-null   object
 2   pop_estimada  5572 non-null   int32 
 3   chave_pop     5572 non-null   object
dtypes: int32(1), object(3)
memory usage: 152.5+ KB


In [77]:
df_pop.to_csv('d_pop.csv', sep = ';', index=False)

# 🎲 DIMENSAO ESTABELECIMENTOS

[Fonte CNES DataSus](https://cnes.datasus.gov.br/pages/downloads/arquivosBaseDados.jsp)

In [78]:
cols = ['CO_CNES','CO_IBGE','NO_RAZAO_SOCIAL','NO_FANTASIA','TP_GESTAO','NU_LATITUDE','NU_LONGITUDE','DS_TURNO_ATENDIMENTO','CO_NATUREZA_JUR','ST_CENTRO_CIRURGICO','ST_CENTRO_OBSTETRICO','ST_CENTRO_NEONATAL','ST_ATEND_HOSPITALAR','ST_ATEND_AMBULATORIAL','CO_AMBULATORIAL_SUS']

planilha_estab = r'C:\Users\carol\OneDrive\Estudos\MBA Data Science\TCC\SINASC\cnes_estabelecimentos.csv'
df_estab = pd.read_csv(planilha_estab, usecols= cols, sep =';', encoding='ISO-8859-1')

In [79]:
#renomeando colunas
novo_nome = {

    'CO_CNES': 'co_cnes',
    'CO_IBGE': 'co_ibge',
    'NO_RAZAO_SOCIAL': 'no_razao_social',
    'NO_FANTASIA': 'no_fantasia',
    'TP_GESTAO': 'tp_gestao',
    'NU_LATITUDE': 'nu_latitude',
    'NU_LONGITUDE': 'nu_longitude',
    'DS_TURNO_ATENDIMENTO': 'ds_turno_atendimento',
    'CO_NATUREZA_JUR': 'co_natureza_jur',
    'ST_CENTRO_CIRURGICO': 'st_centro_cirurgico',
    'ST_CENTRO_OBSTETRICO': 'st_centro_obstetrico',
    'ST_CENTRO_NEONATAL': 'st_centro_neonatal',
    'ST_ATEND_HOSPITALAR': 'st_atend_hospitalar',
    'ST_ATEND_AMBULATORIAL': 'st_atend_ambulatorial',
    'CO_AMBULATORIAL_SUS': 'co_ambulatorial_sus'
    
}

df_estab.rename(columns=novo_nome, inplace=True)

In [80]:
df_estab.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 513211 entries, 0 to 513210
Data columns (total 15 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   co_cnes                513211 non-null  int64  
 1   co_ibge                513211 non-null  int64  
 2   no_razao_social        513209 non-null  object 
 3   no_fantasia            513146 non-null  object 
 4   tp_gestao              513211 non-null  object 
 5   nu_latitude            448229 non-null  float64
 6   nu_longitude           448260 non-null  float64
 7   ds_turno_atendimento   511747 non-null  object 
 8   co_natureza_jur        513207 non-null  float64
 9   st_centro_cirurgico    473356 non-null  float64
 10  st_centro_obstetrico   473356 non-null  float64
 11  st_centro_neonatal     473356 non-null  float64
 12  st_atend_hospitalar    473356 non-null  float64
 13  st_atend_ambulatorial  513211 non-null  float64
 14  co_ambulatorial_sus    513211 non-nu

In [81]:
df_estab['co_natureza_jur'] = df_estab['co_natureza_jur'].fillna(0).astype(int)

col_object = ['co_cnes','co_ibge','no_razao_social','no_fantasia','tp_gestao','ds_turno_atendimento','co_natureza_jur','st_centro_cirurgico','st_centro_obstetrico','st_centro_neonatal','st_atend_hospitalar','st_atend_ambulatorial','co_ambulatorial_sus']
df_estab[col_object] = df_estab[col_object].astype(str)

In [82]:
#nova coluna de natureza do estabelecimento
df_estab['co_natureza_jur_agg'] = df_estab['co_natureza_jur'].str.slice(0,1)

[Link da regra de natureza jurídica](https://concla.ibge.gov.br/estrutura/natjur-estrutura/natureza-juridica-2021.html)

In [83]:
#trazendo a marcação da categoria no df seguinto clasificação do ibge
mapeamento = {'1': 'administracao publica', '2':'entidades empresariais', '3':'entidades sem fins lucrativos', '4':'pessoas físicas', '5': 'organizações internacionais e outras instituições extraterritoriais' }
df_estab['classe_natureza'] = df_estab['co_natureza_jur_agg'].map(mapeamento).fillna('sem info')

In [84]:
df_estab.head().T

,0,1,2,3,4
co_cnes,19,27,35,43,51
co_ibge,260290,260290,260290,260290,260290
no_razao_social,PREFEITURA MUNICIPAL DO CABO DE SANTO AGOSTINHO,CASA DE SAUDE E MATERNIDADE SANTA HELENA LTDA,PREFEITURA MUNICIPAL DO CABO DE SANTO AGOSTINHO,PREFEITURA MUNICIPAL DO CABO DE SANTO AGOSTINHO,PREFEITURA MUNICIPAL DO CABO DE SANTO AGOSTINHO
no_fantasia,POLICLINICA DR JAMACI DE MEDEIROS,CASA DE SAUDE SANTA HELENA,HOSPITAL MENDO SAMPAIO,POLICLINICA DR MANUEL GOMES,POLICLINICA VICENTE MENDES
tp_gestao,M,M,M,M,M
nu_latitude,-8.23166,-8.287,-8.287,-8.28765,-8.287
nu_longitude,-34.969599,-35.035,-35.035,-35.035,-35.035
ds_turno_atendimento,ATENDIMENTO CONTINUO DE 24 HORAS/DIA (PLANTAO:...,ATENDIMENTO CONTINUO DE 24 HORAS/DIA (PLANTAO:...,ATENDIMENTO CONTINUO DE 24 HORAS/DIA (PLANTAO:...,ATENDIMENTOS NOS TURNOS DA MANHA E A TARDE,"ATENDIMENTO NOS TURNOS DA MANHA, TARDE E NOITE"
co_natureza_jur,1244,2062,1244,1244,1244
st_centro_cirurgico,0.0,1.0,1.0,0.0,0.0


In [85]:
df_estab.to_csv('d_estab.csv', sep = ';', index=False)